# Important installation procedures. 


## Necessary python packages 

- pandas
- torch
- matplotlib

You can use conda (recommended) or pip to do this.


## Download the data 
Save it in the project folder </br>  
https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data?select=icml_face_data.csv



## Credits

https://www.kaggle.com/alinaspasskaya/lab-2-face-emotion-classification-with-mlp </br>
https://www.kaggle.com/sharadhaviswanathan/imageclassification-facialexpression


## Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import torch
from torch import nn
from torch.nn import functional as F

## Obtaining and Preprocessing the data

In data preparation the first step will be getting the train, validation and test data and this applies across all the modalities.Preprocessing the dataset varies based on both dataset and the model architecture in place. For example, some model architecture requries certain image dimensions. 


####  Currently in this code we have not done any special preprocessing. Some of the recommended methods that you can check out are to improve the model are

1. Random data augmentation
2. Oversampling or Undersampling the data to makeup for the imbalance in the data


In [ ]:
data_path = "./data/icml_face_data.csv"
data = pd.read_csv(data_path) ## Grayscale images

emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']


def parse_data(data):
    
    image_array = np.zeros(shape=(len(data), 48, 48)) # 1
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48)) # 1
        image_array[i] = image
        
        
    return image_array, image_label

# Splitting the data into train, validation and testing set thanks to Usage column
train_imgs, train_lbls = parse_data(data[data[" Usage"] == "Training"])
val_imgs, val_lbls = parse_data(data[data[" Usage"] == "PrivateTest"])
test_imgs, test_lbls = parse_data(data[data[" Usage"] == "PublicTest"])



def sample_plot(x,y=None):    #x, y are numpy arrays
    n = 10
    samples = random.sample(range(x.shape[0]),n)
    fig, axs = plt.subplots(2,5, figsize=(25,5))
    ax = axs.ravel()
    for i in range(n):
        ax[i].imshow(x[samples[i],:,:], cmap=plt.get_cmap('gray'))
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        if y is not None:
            ax[i].set_title(emotion_labels[y[samples[i]]])

sample_plot(train_imgs,train_lbls)

## Structuring the dataset to pass it through a dataloader in Pytorch

In [ ]:
class FER2013_Dataset(torch.utils.data.Dataset):
    
    def __init__(self,imgs,labels):

        self.imgs = imgs
        self.labels = labels
    
    def __len__(self):
        
        return len(self.imgs)
    
    def __getitem__(self, index):

        img = torch.FloatTensor(self.imgs[index])
        label = self.labels[index]
        return img, label

    
train_dataset = FER2013_Dataset(train_imgs, train_lbls)
valid_dataset = FER2013_Dataset(val_imgs, val_lbls)
test_dataset = FER2013_Dataset(test_imgs, test_lbls)

## Defining the model architecture - A simple Neural Network

In [ ]:
class MLP(nn.Module):
    
    # define model layers
    def __init__(self,img_size,output_size):
        super(MLP, self).__init__()
        self.flatten_img_size = img_size[0]*img_size[1]
        self.flatten_fc = nn.Linear(self.flatten_img_size,120)
        self.fc1 = nn.Linear(120,84)
        self.output_layer = nn.Linear(84,output_size)
        
    
    def forward(self, img):
        img = img.view(-1,self.flatten_img_size) ## img size: Batch size x 48 x 48 --> Batch size x 2304
        
        embedding = self.flatten_fc(img) ## Batch size x 2304 --> Batch size x 120
        
        embedding = F.relu(embedding) ## Applying non-linear activation after a dense layer
        
        embedding =  F.relu(self.fc1(embedding))  ## Batch size x  120--> Batch size x 84
        
        logits =  self.output_layer(embedding)    ## Batch size x 84 --> Batch size x output size
         
        return logits



## Defining the model architecture - A basic CNN classifier

In [ ]:
class CNN_Net(nn.Module):

    def __init__(self):
        super(CNN_Net, self).__init__()

        # kernel
        self.conv1 = nn.Conv2d(1,6,1) ## in channel, out channel
        self.conv2 = nn.Conv2d(6,16,6) ## in channel, out channel
   
        self.fc1 = nn.Linear(16*9*9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)

    def forward(self, x):
        
        x = x.unsqueeze(1)

        x = F.relu(self.conv1(x))
        
        x = F.max_pool2d(x, (2, 2))

        x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


 <font color='red'> <b> >> CHANGE THE PARAMETERS HERE AND RE-RUN FROM THIS CELL</font>

## Getting things ready for training

A fairly general checklist,

1. Choose the model
2. Initialise the data loaders using the created dataset
3. Loss function
4. Optimiser
5. Learning rate scheduler (You can try adding this)
6. Hyperparmeter settings
 

###### Do note that the choice of the items in this checklist is highly dependant on your choice of task, model and data.


In [ ]:
nepoch = 10
learning_rate = 0.001
batch_size = 64
model_arch = "mlp"

In [ ]:
## SET THE SEED FOR GETTING THE SAME RESULTS AGAIN
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# MODEL
if model_arch == "mlp":
    model = MLP((48,48),7)
elif model_arch == "cnn_net":
    model = CNN_Net()
    
# DATA LOADER

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size,shuffle= True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size = 1,shuffle= False)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = 1,shuffle= False)

# LOSS
loss_fn = nn.CrossEntropyLoss()  ## applies softmax to the logits before computing the loss 


# OPTIMISER
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)



## Training

The training dataset is divided into batches by the dataloader. The weights of the model are updated after learning every batch. Here, we choose the best model based on the validation performance (accuracy). The saved best model is used to get the test accuracy.

In [ ]:
for epoch in range(nepoch):  # loop over the dataset multiple times
    train_loss = 0
    train_acc = 0
    best_criterion = 0
    
    ## model in training mode
    model.train()
    for i, data in enumerate(train_loader):
        
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        pred = torch.argmax(outputs,dim=1)
       
        num_correct = (pred==labels).sum().item()
        train_acc += num_correct/inputs.size()[0]
   
    
    val_acc = 0
    
    ## model in evaluation mode
    model.eval()
        
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data
            
            outputs = model(inputs)
            
            pred = torch.argmax(outputs,dim=1)
       
            num_correct = (pred==labels).sum().item()
        
            val_acc += num_correct/inputs.size()[0]
            
    print("Epoch:{} Train Loss:{:.2f} Train Accuracy:{:.2f} Val Accuracy:{:.2f}".format(epoch,train_loss/len(train_loader),(train_acc/len(train_loader))*100,(val_acc/len(valid_loader))*100))
    
    if val_acc > best_criterion:
        torch.save(model.state_dict(),"./{}_model_best.pth.tar".format(model_arch))
        
    
    best_criterion = max(best_criterion,val_acc)
    

    

In [ ]:
if model_arch == "mlp":
    model = model = MLP((48,48),7)
elif model_arch == "cnn_net":
    model = CNN_Net()

model.load_state_dict(torch.load("./{}_model_best.pth.tar".format(model_arch)))
model.eval()

test_acc = 0
total = 0
model.eval()

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data

        outputs = model(inputs)

        pred = torch.argmax(outputs,dim=1)

        num_correct = (pred==labels).sum().item()

        test_acc += num_correct/inputs.size()[0]

print("Test Accuracy: {:.2f}".format((test_acc/len(test_loader))*100))


## This performance can be further improved by transfer learning which will be explained in detailed in the upcoming tutorial. Stay tuned !!